In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score

In [31]:
filename = 'test.txt'
file = open(filename, 'rt')
text = file.read()
file.close()

def extract_statement(tags):
    '''Extracts a (subject, relation, object) triple from each statement based on the POS tags'''
    subject, relation, obj = '', '', ''
    for word,tag in tags:
        if tag == 'NNP':
            subject = word
        elif tag == 'VBD' or word == 'journeyed': # TODO: 'journeyed' is tagged improperly
            relation = word
        if tag == 'NNP' or tag == 'NN':
            obj = word
    return (subject, relation, obj)
    



    
    # filter out stop words
#     stop_words = set(stopwords.words('english'))
#     words = [w for w in words if not w in stop_words]
#     print(words)





('', '', 'cash')
('', '', 'fee')
('', '', 'patient')
('', '', 'insurance')
('', '', 'fee')
('', '', 'fee')
('', '', 'desk')
('', '', 'attention')
('', '', 'hospital')
('', '', 'system')
('', '', '')
('', '', '')
('', '', '')
('', '', 'office')


In [1]:
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import string

sent_tokenize_list = sent_tokenize(response.cleaned_text)
print(len(sent_tokenize_list))
print(sent_tokenize_list[1])
setence = Sentence(sent_tokenize_list[1])

# split into sentences

stop_words = set(stopwords.words('english'))
sentences = sent_tokenize(text)
for sentence in sentences:
    
    tokens = word_tokenize(sentence)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    tagged_word = nltk.pos_tag(words)
    print(extract_statement(tagged_word))
    




NameError: name 'text' is not defined

In [ ]:
sent_tokenize_list = sent_tokenize(response.cleaned_text)
print(len(sent_tokenize_list))
print(sent_tokenize_list[1])
setence = Sentence(sent_tokenize_list[1])

In [22]:
import textrazor
from collections import Counter
from nltk.tokenize import sent_tokenize

# textrazor.api_key = "6539e01f2076cebb1be13a160c17beca2d4e681b6236fc0e600e0220"
# client = textrazor.TextRazor(extractors=["entities", "topics","entailments","relations", "words", "phrases", "dependency-trees"])

# client.set_cleanup_mode("cleanHTML")
# client.set_cleanup_return_cleaned(return_cleaned=True)
# response = client.analyze_url("http://wenzhang.16fan.com/a/248105.html")
client = textrazor.TextRazor(api_key="6539e01f2076cebb1be13a160c17beca2d4e681b6236fc0e600e0220", extractors=["words", "phrases"])
activity_name = "apply for visa" 

response = client.analyze(activity_name)

   
for np in response.noun_phrases():
    print(activity_name[0:np.words[0].input_start_offset])
    print(type(activity_name[np.words[0].input_start_offset: np.words[-1].input_end_offset]))

# to_analyze = """post offices in Japan offer a variety of services including the collection and sending of postal items such as letters, parcels, post cards, postal orders and registered mail both within Japan and overseas."""
# response = client.analyze(to_analyze)
# for sentence in response.sentences():
#     for word in sentence.words:
#         for rel in word.relations:
#             print([pword.token for pword in rel.predicate_words])
                
# c = Counter(response.noun_phrases())
# c = list(c)
# print(c)



# for np in response.entities():
#     print(np.id)
    

# generate [word list] Relation part 
# for rel in response.relations():
#     for param in rel.params:
#         print([word.token for word in param.param_words],param.relation)

        

        
        
#         print(param.relation)
#         for word in param.relation:
#             print(word.token, word.relation)

# for rel in response.relations():
#     for pword in rel.predicate_words:
#         print(pword.token)

# entity_list = [entity.id  for entity in response.entities() if not entity.id.isdigit()]
# c = Counter(entity_list)
# c = list(c)
# print(c)

apply for 
<class 'str'>


In [54]:
import textrazor
from neo4j.v1 import GraphDatabase, basic_auth
driver = GraphDatabase.driver('bolt://localhost:7687', auth=basic_auth('neo4j', 'iwate'))
session = driver.session()
temp = ""


def find_how_to_do(activity_name):
    '''Find the related rel and entities in the knowledge graph'''

    query = '''
        MATCH (s:Activity)<-[r:REL]-(entity:Entity) where s.name contains $name
        RETURN s.name,r.name,entity.name 
    '''
    
    print(activity_name)
    
    
    query_static = '''
    match (act:Activity{name:"apply for"})-[:Apply_to]->(en:Entity{name:"visa"})
        with en as entity,act as activity 
        match (res:Entity)<-[:result]-(entity)-[:require]->(req:Entity) 
        return activity,entity,req,res
    
    '''
    query_try = '''
        match (act:Activity{name:$act_name})-[:Apply_to]->(en:Entity{name:$en_name})
        with en as entity,act as activity 
        match (res:Entity)<-[:result]-(entity)-[:require]->(req:Entity) 
        return activity,entity,req,res
    
    '''
#     global temp
#     str_result = ""
    client = textrazor.TextRazor(api_key="6539e01f2076cebb1be13a160c17beca2d4e681b6236fc0e600e0220",
                                 extractors=["words", "phrases"])

    response = client.analyze(activity_name)

    for np in response.noun_phrases():
        action_name = activity_name[0:np.words[0].input_start_offset]
        entity_name = activity_name[np.words[0].input_start_offset: np.words[-1].input_end_offset]
        
    print(str(action_name))
    print(entity_name)
    print(session.run(query_static).single())
    record = session.run(query_static).single()
    print(record["activity"]["name"])
    print(session.run(query_try, act_name="apply for", en_name="visa").single())
        
    return 
#     for recode in session.run(query_try, act_name=action_name, en_name=entity_name):
#         temp += recode["activity.name"]+" "+recode["entity.name"]+"  require  "+recode["req.name"]+"  result  "+recode["res.name"]
#         # print(recode["s.name"]+"    "+recode["r.name"]+"    "+recode["entity.name"]+"\n")
#     return temp

          
          
find_how_to_do("apply for visa")


apply for visa
apply for 
visa
<Record activity=<Node id=30 labels={'Activity'} properties={'name': 'apply for'}> entity=<Node id=6 labels={'Entity'} properties={'name': 'visa'}> req=<Node id=22 labels={'Entity'} properties={'name': 'passport'}> res=<Node id=6 labels={'Entity'} properties={'name': 'visa'}>>
apply for
<Record activity=<Node id=30 labels={'Activity'} properties={'name': 'apply for'}> entity=<Node id=6 labels={'Entity'} properties={'name': 'visa'}> req=<Node id=22 labels={'Entity'} properties={'name': 'passport'}> res=<Node id=6 labels={'Entity'} properties={'name': 'visa'}>>


In [72]:
from bs4 import BeautifulSoup
import requests
import json
import textrazor
from py2neo import authenticate, Graph

USERNAME = "neo4j"  # use your actual username
PASSWORD = "iwate"  # use your actual password
authenticate("localhost:7474", USERNAME, PASSWORD)
graph = Graph()
activity_name = "apply for visa"


def find_how(activity_name):
    query = """
    match (act:Activity{name:{act_name}})-[:Apply_to]->(en:Entity{name:{en_name}})
            with en as entity,act as activity 
            match (res:Entity)<-[:result]-(entity)-[:require]->(req:Entity) 
            return activity.name,entity.name,req.name,res.name
    
    """
    client = textrazor.TextRazor(api_key="6539e01f2076cebb1be13a160c17beca2d4e681b6236fc0e600e0220",
                                     extractors=["words", "phrases"])
    
    response = client.analyze(activity_name)
    #Do not foget to strip all the phrase/
    for np in response.noun_phrases():
        action_name = activity_name[0:np.words[0].input_start_offset]
        entity_name = activity_name[np.words[0].input_start_offset: np.words[-1].input_end_offset]
    
    cursor = graph.run(query, act_name=action_name.strip(), en_name=entity_name.strip())
    records = []
    for record in cursor:
        records.append(record)
    return records[0]['activity.name'] + " " + records[0]['entity.name'] + " require " + records[0]['req.name'] + " result " + records[0]['res.name']

print(find_how(activity_name))

apply for visa require passport result visa


In [16]:

temp = """Work and Study\n\n\n\nImmigration\nAbout different types of visa.\n\nWorking Holiday\nA visa for young people from certain countries.\n\nWorking in Japan\nGeneral information about working in Japan.\n\nTeaching Languages\nNative language teachers are still in high demand.\n\nStudying in Japan\nGeneral information about studying in Japan.\n\nLanguage Schools\nA list of Japanese language schools.\n\n\n\n\nAccommodation\n\n\n\nGuesthouses\nInexpensive accommodation for foreigners.\n\nFinding an apartment\nHow to look for an apartment in Japan.\n\nJapanese Apartments\nGeneral information about Japanese apartments.\n\nUtilities\nInformation about gas, water and electricity.\n\nElectricity\nAbout voltage and Japanese plugs.\n\nFurniture\nIntroduction of some unique Japanese furniture.\n\nGarbage Disposal\nHow to properly get rid of your garbage.\n\n\n\n\nEveryday Life\n\nFinancial Matters\n\n\n\nTaxes\nInformation about paying taxes in Japan.\n\nCost of Living\nInformation about prices in Japan.\n\nBanking\nAbout Japanese banks and how to open an account.\n\n\n\nBasic Services\n\n\n\nPostal Services\nAbout post offices and Japanese addresses.\n\nDelivery Services\nAbout door-to-door delivery services (takuhaibin).\n\nTelephones\nA guide to phones in Japan.\n\nMobile Phones\nAbout cell phones.\n\n\n\nTransportation\n\n\n\nTransportation\nAbout transportation in Japan.\n\nDriving in Japan\nAbout driving and owning a car in Japan.\n\nBicycles\nAbout everyday use of bicycles.\n\n\n\nShopping\n\n\n\nSupermarkets\nAbout Japanese supermarkets.\n\n100 Yen Shops\nAbout 100 Yen Shops.\n\nConvenience Stores\nAbout Japanese convenience stores.\n\nDepartment Stores\nAbout Japanese department stores.\n\n\n\nVarious Issues\n\n\n\nHolidays\nA list of public holidays and annual events.\n\nCalendar\nBasic information on the Japanese calendar.\n\nEmergencies\nWhat to do in case of an emergency.\n\nNames\nBasic information on Japanese names.\n\nBusiness cards\nBusiness cards are used extensively in Japan.\n\n\n\nGeneral Background\n\n\n\nFood\nAn introduction to the Japanese cuisine.\n\nEtiquette\nIntroduction to Japanese customs.\n\nLanguage\nIntroduction to the Japanese language.\n\nReligion\nIntroduction to religion in Japan.\n\nTravel\nIntroduction on travel in Japan.""
temp.split("""\n\n\n\n""")

SyntaxError: unexpected character after line continuation character (<ipython-input-16-87316363c7e5>, line 3)

In [133]:
from nltk.tokenize import sent_tokenize
temp = """International Driving Permit
(issued by signatory countries
of the Geneva Convention)
If you have an International
Driving Permit issued by a
signatory country of the Geneva
Convention, you are allowed to
drive in Japan during the period
specified below.
Period: The shorter of either:
one year from the date on which
you landed in Japan; or until the
expiration day of your
International Driving Permit.
This provision is not applicable
if you are registered on the
Basic Resident Register, left
Japan with a reentry permit and
landed in Japan again in less
than three months after leaving
Japan."""

sent_tokenize_list = sent_tokenize(temp)
test = sent_tokenize_list[1].replace('\n', ' ')
print(test)



Period: The shorter of either: one year from the date on which you landed in Japan; or until the expiration day of your International Driving Permit.


In [16]:
from collections import Counter
x = Counter({'a':5, 'b':3, 'c':7})
X = x.most_common()
print([x[0] for x in X])


['c', 'a', 'b']


In [18]:
rules_str = """
% Match two companies in a 'buy' relation.
acquisition_rumor(CompanyA, CompanyB, EntailedWord) :-
    entity_type(CompanyA, 'Organization'),
    entity_type(CompanyB, 'Service'),
    relation_overlap(BuyRelation, 'SUBJECT', CompanyA, 'OBJECT', CompanyB),
    entailment_overlap(_, BuyRelation, EntailedWord),
    member(EntailedWord, ['use', 'offer', 'provide']).
"""
client = textrazor.TextRazor(extractors=["acquisition_rumor"])
client.set_rules(rules_str)
# client.set_cleanup_mode("cleanHTML")
client.set_cleanup_return_cleaned(return_cleaned = True)
response = client.analyze_url("https://www.japanvisitor.com/japan-travel/japan-residents/japan-post")
# print(response.cleaned_text)
response = client.analyze(response.cleaned_text)
print(response.matching_rules())

# for rumor in response.acquisition_rumor:
#     print(rumor)

[]


In [125]:
import textrazor





textrazor.api_key = "6539e01f2076cebb1be13a160c17beca2d4e681b6236fc0e600e0220"

client = textrazor.TextRazor(extractors=["entities", "topics","entailments","relations", "words", "phrases", "dependency-trees"])
response = client.analyze("""Japanese post offices also offer banking and insurance.""")



# print(len(response.relations()))
words = response.words()
rels = response.relations()
props = response.properties()



for rel in rels:
    for words in rel.predicate_words:
        print(word.token)
            

#     print(predict.token)

# print(len(props))
# print(props[0].predicate_words)


# print(rels[0].params[1].relation)
# print(rels[0].params[1].param_words)

for rel in rels:
    for param in rel.params:
        for word in param.param_words:
            print(word.token)
            
            
            
#     for word in rel.params[1].param_words:
#     print(word.token)

# for rel in rels:
#     print(rel.params)

overseas
overseas


In [117]:
import spacy
from textpipeliner import PipelineEngine, Context
from textpipeliner.pipes import *

nlp = spacy.load("en")
# doc = nlp("""The approximately 24,000 post offices in Japan offer a variety of services including the collection and sending of postal items such as letters, parcels, post cards, postal orders and registered mail both within Japan and overseas. Japanese post offices also offer banking and insurance.""")
doc = nlp("""Japanese post offices also offer banking and insurance.""")


pipes_structure = [SequencePipe([FindTokensPipe("VERB/nsubj/*"),
                                 NamedEntityFilterPipe(),
                                 NamedEntityExtractorPipe()]),
                   FindTokensPipe("VERB"),
                   AnyPipe([SequencePipe([FindTokensPipe("VBP/dobj/NNP"),
                                          AggregatePipe([NamedEntityFilterPipe("GPE"), 
                                                NamedEntityFilterPipe("PERSON")]),
                                          NamedEntityExtractorPipe()]),
                            SequencePipe([FindTokensPipe("VBP/**/*/pobj/NNP"),
                                          AggregatePipe([NamedEntityFilterPipe("LOC"), 
                                                NamedEntityFilterPipe("PERSON")]),
                                          NamedEntityExtractorPipe()])])]

engine = PipelineEngine(pipes_structure, Context(doc), [0,1,2])
engine.process()

[]

In [ ]:

# curl -X POST \
#     -H "x-textrazor-key: YOUR_API_KEY" \
#     -d "extractors=entities,entailments" \
#     -d "text=Spain's stricken Bankia expects to sell off its vast portfolio of industrial holdings that includes a stake in the parent company of British Airways and Iberia." \
#     https://api.textrazor.com/
# import requests

# url = 'https://api.textrazor.com/'

# data = {'message':'<messageML>Hello world!</messageML>'}
# r = requests.post(url, files=data)

# print(r.text)

curl -X POST \
    -H "x-textrazor-key: 6539e01f2076cebb1be13a160c17beca2d4e681b6236fc0e600e0220" \
    -d "extractors=entities,entailments" \
    -d "text=Spain's stricken Bankia expects to sell off its vast portfolio of industrial holdings that includes a stake in the parent company of British Airways and Iberia." \
    https://api.textrazor.com/


import requests

headers = {
    'X-TextRazor-Key': '6539e01f2076cebb1be13a160c17beca2d4e681b6236fc0e600e0220'
}

payload = (('extractors', 'entailments'), ('extractors', 'entities'), ('text',"Spains stricken Bankia expects to sell off its vast portfolio of industrial holdings that includes a stake in the parent company of British Airways and Iberia."))

t = requests.post('url', headers=headers, data=payload).json()
print(t.text)

In [3]:
import requests

headers = {
    'x-textrazor-key': '6539e01f2076cebb1be13a160c17beca2d4e681b6236fc0e600e0220',
}

data = [
  ('extractors', 'entities,entailments'),
  ('text', 'Spain\'s stricken Bankia expects to sell off its vast portfolio of industrial holdings that includes a stake in the parent company of British Airways and Iberia.'),
]

response = requests.post('https://api.textrazor.com/', headers=headers, data=data).json()


test
{'response': {'sentences': [{'position': 0, 'words': [{'position': 0, 'startingPos': 0, 'endingPos': 5, 'stem': 'spain', 'lemma': 'spain', 'token': 'Spain', 'partOfSpeech': 'NNP'}, {'position': 1, 'startingPos': 5, 'endingPos': 7, 'stem': "'s", 'lemma': "'s", 'token': "'s", 'partOfSpeech': 'POS'}, {'position': 2, 'startingPos': 8, 'endingPos': 16, 'stem': 'stricken', 'lemma': 'stricken', 'token': 'stricken', 'partOfSpeech': 'JJ'}, {'position': 3, 'startingPos': 17, 'endingPos': 23, 'stem': 'bankia', 'lemma': 'bankia', 'token': 'Bankia', 'partOfSpeech': 'NNP'}, {'position': 4, 'startingPos': 24, 'endingPos': 31, 'stem': 'expect', 'lemma': 'expect', 'token': 'expects', 'partOfSpeech': 'VBZ'}, {'position': 5, 'startingPos': 32, 'endingPos': 34, 'stem': 'to', 'lemma': 'to', 'token': 'to', 'partOfSpeech': 'TO'}, {'position': 6, 'startingPos': 35, 'endingPos': 39, 'stem': 'sell', 'lemma': 'sell', 'token': 'sell', 'partOfSpeech': 'VB'}, {'position': 7, 'startingPos': 40, 'endingPos': 43,